In [5]:
import pandas as pd
import re  # The re module in Python is primarily used for matching patterns in strings
from collections import Counter
import math

def preprocess_text(text):
    return re.sub(r'\W+', ' ', text.lower())

# Assuming the file path is correct
excel_data = pd.read_excel("C:\\Users\\91939\\Dropbox\\Sem 5\\NLP\\LAB\\SASTRA University.xlsx")
all_text = ' '.join(excel_data.iloc[:, 0].apply(preprocess_text))

# Tokenize and count word frequencies
words = all_text.split()
word_freq = Counter(words)
pair_freq = Counter(zip(words, words[1:]))

def t_test(word1, word2, word_freq, pair_freq):
    f_w1 = word_freq[word1]
    f_w2 = word_freq[word2]
    f_w1w2 = pair_freq[(word1, word2)]

    # Mean and standard deviation calculations
    mean_w1 = f_w1w2
    mean_w2 = (f_w1 + f_w2) / 2
    std_w1 = math.sqrt(f_w1w2 * (1 - (f_w1w2 / f_w1))) if f_w1 > 0 else 0
    std_w2 = math.sqrt(f_w1w2 * (1 - (f_w1w2 / f_w2))) if f_w2 > 0 else 0

    # Check and calculate t-test statistic
    if f_w1 > 0 and f_w2 > 0 and std_w1 > 0 and std_w2 > 0:
        t_stat = (mean_w1 - mean_w2) / math.sqrt((std_w1**2 / f_w1) + (std_w2**2 / f_w2))
        return t_stat
    return 0

# Define words to test
word1, word2 = 'sastra', 'university'
t_stat = t_test(word1, word2, word_freq, pair_freq)

# Evaluate collocation based on t-test statistic
if abs(t_stat) > 2.576:
    print('SASTRA University forms a collocation')
else:
    print('SASTRA University does not form a collocation')

print(f"T-test statistic for '{word1} {word2}': {t_stat}")


SASTRA University forms a collocation
T-test statistic for 'sastra university': -18.590825088229


In [7]:
import pandas as pd
import numpy as np
from collections import Counter

# Load data from the Excel file
df = pd.read_excel("C:\\Users\\91939\\Dropbox\\Sem 5\\NLP\\LAB\\SASTRA University.xlsx")

# Tokenization function
def s_tokenize(text):
    return text.lower().split()

# Count bigrams and unigrams
bigram_counts = Counter()
unigram_counts = Counter()
for sentence in df.iloc[:, 0]:  # Assuming the first column contains sentences
    tokens = s_tokenize(sentence)
    unigram_counts.update(tokens)
    for i in range(len(tokens) - 1):
        bigram = (tokens[i], tokens[i + 1])
        bigram_counts[bigram] += 1

# Total number of words
N = sum(unigram_counts.values())

# Chi-square test function
def chi_2_text(w1, w2, c1, c2, c12):
    # Observed frequencies
    a = c12
    b = c1 - a
    cx = c2 - a
    d = N - (c1 + c2 - c12)
    
    # Expected frequencies based on independence
    e_w1 = (c1 * c2) / N
    e_w2 = (c1 * (N - c2)) / N
    e_w3 = ((N - c1) * c2) / N
    e_w4 = ((N - c1) * (N - c2)) / N
    
    # Chi-square computation
    chi_value = (
        ((a - e_w1)**2 / e_w1 if e_w1 > 0 else 0) +
        ((b - e_w2)**2 / e_w2 if e_w2 > 0 else 0) +
        ((cx - e_w3)**2 / e_w3 if e_w3 > 0 else 0) +
        ((d - e_w4)**2 / e_w4 if e_w4 > 0 else 0)
    )
    
    return {'chi_value': chi_value, 'word1': w1, 'word2': w2}

# Example usage
word1, word2 = 'sastra', 'university'
c1 = unigram_counts[word1]
c2 = unigram_counts[word2]
c12 = bigram_counts[(word1, word2)]

result = chi_2_text(word1, word2, c1, c2, c12)
print(f"Chi-square value for '{word1} {word2}':", result['chi_value'])
if result['chi_value'] > 3.841:
    print(f"'{word1} {word2}' forms a collocation with chi-square value {result['chi_value']: }")
else:
    print(f"'{word1} {word2}' does not form a collocation with chi-square value {result['chi_value']: }")

Chi-square value for 'sastra university': 345.40016059957173
'sastra university' forms a collocation with chi-square value  345.40016059957173
